In [1]:
# Force the local gqcpy to be imported
import sys
sys.path.insert(0, '../../build/gqcpy/')

import gqcpy
import numpy as np

In [2]:
neutral = gqcpy.Molecule.ReadXYZ("../../gqcp/tests/data/h2o_crawdad.xyz" , 0)  # create a neutral molecule
ion_pos = gqcpy.Molecule.ReadXYZ("../../gqcp/tests/data/h2o_crawdad.xyz" , +1)  # create a an ion with a depleted electron
ion_neg = gqcpy.Molecule.ReadXYZ("../../gqcp/tests/data/h2o_crawdad.xyz" , +1)  # create a an ion with an extra electron

In [3]:
def C_RHF(molecule):
    
    N = molecule.numberOfElectrons()
    N_P = molecule.numberOfElectronPairs()
    
    spin_orbital_basis = gqcpy.RSpinOrbitalBasis_d(molecule, "STO-3G")
    K = spin_orbital_basis.numberOfSpatialOrbitals()
    print("N: {}\nN_P: {}\nK: {}".format(N, N_P, K))

    hamiltonian = gqcpy.RSQHamiltonian.Molecular(spin_orbital_basis, molecule)
    S = spin_orbital_basis.quantizeOverlapOperator().parameters()

    environment = gqcpy.RHFSCFEnvironment.WithCoreGuess(N, hamiltonian, S)
    solver = gqcpy.RHFSCFSolver.Plain()
    objective = gqcpy.DiagonalRHFFockMatrixObjective(hamiltonian)  # use the default threshold of 1.0e-08
    rhf_parameters = gqcpy.RHF.optimize(objective, solver, environment).groundStateParameters()
    
    return rhf_parameters.expansion()

In [4]:
def C_UHF(molecule):
    N = molecule.numberOfElectrons()
    N_alpha = N//2 + 1 if N%2 else N//2
    N_beta = N//2

    spinor_basis = gqcpy.RSpinOrbitalBasis_d(molecule, "STO-3G")
    K = spinor_basis.numberOfSpatialOrbitals()
    print("N_alpha: {}\nN_beta: {}\nK: {}".format(N_alpha, N_beta, K))
    
    hamiltonian = gqcpy.RSQHamiltonian.Molecular(spinor_basis, molecule) 
    S = spinor_basis.quantizeOverlapOperator().parameters()
    
    environment = gqcpy.UHFSCFEnvironment.WithCoreGuess(N_alpha, N_beta, hamiltonian, S)
    solver = gqcpy.UHFSCFSolver.Plain()
    uhf_parameters = gqcpy.UHF.optimize(solver, environment).groundStateParameters()
    return uhf_parameters.expansion()

In [5]:
print("neutral molecule: ")
C_neutral = C_UHF(neutral)
print("\n\nion:")
C_ion = C_UHF(ion_pos)

neutral molecule: 
N_alpha: 5
N_beta: 5
K: 7


ion:
N_alpha: 5
N_beta: 4
K: 7


In [6]:
def TM(C_ion, C_neutral):
    return C_ion.inverse().transformed(C_neutral)

In [7]:
U_alpha = TM(C_ion.alpha, C_neutral.alpha)
U_beta = TM(C_ion.beta, C_neutral.beta)

U = TM(C_ion, C_neutral)

In [8]:
print("Is C_ion unitary? ", C_ion.isUnitary(1e-12))
print("Is C_neutral unitary? ", C_neutral.isUnitary(1e-12))
print("Is U unitary? ", U.isUnitary(1e-12))

Is C_ion unitary?  False
Is C_neutral unitary?  False
Is U unitary?  True


In [9]:
K = 7

onv_basis_I = gqcpy.SpinResolvedONVBasis(K, 5, 4)  # number of spatial orbitals, number of alpha-electrons, number of beta-electrons
onv_basis_J = gqcpy.SpinResolvedONVBasis(K, 5, 5)  # number of spatial orbitals, number of alpha-electrons, number of beta-electrons

In [10]:
Psi_I = gqcpy.LinearExpansion_SpinResolved.HartreeFock(onv_basis_I)
#Psi_I.coefficients()
Psi_J = gqcpy.LinearExpansion_SpinResolved.HartreeFock(onv_basis_J)
#Psi_J.coefficients()

In [11]:
# Andere namen verzinnen voor de componenten later.
DM_I = Psi_I.calculateSpinResolved1DM()
DM_I_OD = DM_I.orbitalDensity()

DM_J = Psi_J.calculateSpinResolved1DM()
DM_J_OD = DM_J.orbitalDensity()

In [12]:
DM_I_transformed = DM_I.transformed(U)

fukui_matrix_neg = (DM_J - DM_I).orbitalDensity()
eigenvalues, eigenvectors = np.linalg.eigh(fukui_matrix_neg)
eigenvectors

array([[1., 0., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 0., 0., 1., 0.]])

In [14]:
(DM_J - DM_I).orbitalDensity()

array([[0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0.]])